# FLUX-Schnell Inference

This notebook lets you:

2. Save/load embeddings
5. Generate images with FLUX using modified embeddings

```mermaid
flowchart LR
    subgraph Embeddings
        T5[T5 Embedding<br/>JSON<br/>512 × 4096]
        CLIP[CLIP Embedding<br/>JSON<br/>77 × 768]
    end

    subgraph FLUX Pipeline
        T5E[T5 Prompt<br/>Embeds]
        PE[Pooled<br/>Embeds]
        DT[Diffusion<br/>Transformer]
        VAE[VAE<br/>Decoder]
    end

    T5 --> T5E
    CLIP -->|EOS token| PE
    
    T5E -->|cross-attention| DT
    PE -->|global conditioning| DT
    
    DT --> VAE --> IMG[Generated Image]
```

## Installation

Run this cell first to install required packages:

In [2]:
import torch
import json
import numpy as np
from transformers import T5EncoderModel, T5Tokenizer
from diffusers import FluxPipeline
import ipywidgets as widgets
from IPython.display import display, Image as IPImage
from PIL import Image
import os
from pathlib import Path

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load models path from config
current_dir = Path.cwd()
models_path_file = current_dir.parent / "misc/paths/models.txt"
with open(models_path_file, 'r') as f:
    models_path = f.read().strip()
MODELS_DIR = current_dir.parent / models_path

T5_MODEL_PATH = os.path.join(MODELS_DIR, "t5-v1_1-xxl")
CLIP_MODEL_PATH = os.path.join(MODELS_DIR, "clip")
FLUX_MODEL_PATH = os.path.join(MODELS_DIR, "FLUX.1-schnell")

os.makedirs(MODELS_DIR, exist_ok=True)
print(f"Models directory: {os.path.abspath(MODELS_DIR)}")
print(f"T5 path: {os.path.abspath(T5_MODEL_PATH)}")
print(f"FLUX path: {os.path.abspath(FLUX_MODEL_PATH)}")

Using device: cuda
Models directory: /shares/weddigen.ki.uzh/laura_wagner/latent_vandalism_workshop/data/models
T5 path: /shares/weddigen.ki.uzh/laura_wagner/latent_vandalism_workshop/data/models/t5-v1_1-xxl
FLUX path: /shares/weddigen.ki.uzh/laura_wagner/latent_vandalism_workshop/data/models/FLUX.1-schnell


### Download Flux schnell from huggingface

FYI: if you don't have the model locally in ../data/models/FLUX.1-schnell, you have to download it manually from huggingface, because it is in a gated repository, put your huggingface access token in ../misc/credentials/hf.txt to download the model from the repository. THis process only works if you have previously requested access to the repository from huggingface and have generated the access token with the right permissions

In [3]:
# Load Hugging Face token from file
from pathlib import Path

# Get the token file path
current_dir = Path.cwd()
token_file = current_dir.parent / "misc/credentials/hf.txt"

print(f"Looking for HF token at: {token_file}")

if token_file.exists():
    with open(token_file, 'r') as f:
        hf_token = f.read().strip()
    
    # Set the token as an environment variable
    os.environ['HF_TOKEN'] = hf_token
    
    # Also login using huggingface_hub
    from huggingface_hub import login
    login(token=hf_token)

Looking for HF token at: /shares/weddigen.ki.uzh/laura_wagner/latent_vandalism_workshop/misc/credentials/hf.txt


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
# Load FLUX
try:
    if not os.path.exists(FLUX_MODEL_PATH):
        flux_pipe = FluxPipeline.from_pretrained(
            "black-forest-labs/FLUX.1-schnell",
            torch_dtype=torch.bfloat16
        )
        flux_pipe.save_pretrained(FLUX_MODEL_PATH)
    else:
        flux_pipe = FluxPipeline.from_pretrained(
            FLUX_MODEL_PATH,
            torch_dtype=torch.bfloat16,
            local_files_only=True
        )
    
    flux_pipe = flux_pipe.to(device)
    
except Exception as e:
    print(f"Error loading FLUX: {e}")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Generate Image from embedding

In [6]:
# Setup directories
T5_EMBEDDINGS_DIR = current_dir.parent / "data/embeddings/T5"
CLIP_EMBEDDINGS_DIR = current_dir.parent / "data/embeddings/CLIP"

# Global variables for loaded embeddings
loaded_t5_embedding = None
loaded_clip_embedding = None
loaded_t5_prompt = None
loaded_clip_prompt = None

# Setup output directory
OUTPUT_IMAGES_DIR = current_dir.parent / "output/images"
os.makedirs(OUTPUT_IMAGES_DIR, exist_ok=True)

In [11]:
# ==================== HELPER FUNCTIONS ====================

def parse_embedding_filename(filename):
    """
    Parse embedding filename to extract tokens and manipulation.
    Returns (tokens_string, manipulation_string)
    
    Examples:
    - 'a_red_cat_sitting.json' -> ('aredcatsitting', '')
    - 'a_red_cat_sitting_scaled_2.0x.json' -> ('aredcatsitting', 'scaled')
    - 'an_elephant_inverted.json' -> ('anelephant', 'inverted')
    - 'an_elephant_zeroed_30pct.json' -> ('anelephant', 'zeroed')
    - 'examples/a_red_cat.json' -> ('aredcat', '')  # handles subdirectories
    """
    if not filename:
        return ('unknown', '')
    
    # Extract just the filename (strip directory path)
    basename = os.path.basename(filename)
    
    # Remove .json extension
    base_name = basename.rsplit('.', 1)[0]
    
    # Split by underscore
    parts = base_name.split('_')
    
    # First 4 parts are the tokens - join without underscores
    if len(parts) <= 4:
        tokens = ''.join(parts)
        return (tokens, '')
    
    tokens = ''.join(parts[:4])
    
    # Get manipulation type (simplified)
    # Just take the first part after the tokens
    manipulation_parts = parts[4:]
    if manipulation_parts:
        # Get the manipulation type (first word after tokens)
        manipulation = manipulation_parts[0]
    else:
        manipulation = ''
    
    return (tokens, manipulation)

# ==================== T5 EMBEDDING WIDGETS ====================

# T5 embedding selection
t5_file_dropdown = widgets.Dropdown(
    options=[],
    description='T5 Embedding:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

load_t5_button = widgets.Button(
    description='Load T5',
    button_style='success'
)

t5_load_output = widgets.Output()

# Populate T5 dropdown options (including subfolders)
if T5_EMBEDDINGS_DIR.exists():
    t5_files = sorted([str(f.relative_to(T5_EMBEDDINGS_DIR)) for f in T5_EMBEDDINGS_DIR.glob('**/*.json')])
    t5_file_dropdown.options = t5_files

def load_t5_embedding_file(b):
    global loaded_t5_embedding, loaded_t5_prompt
    
    with t5_load_output:
        t5_load_output.clear_output()
        
        filename = t5_file_dropdown.value
        if not filename:
            print("❌ No file selected!")
            return
        
        filepath = T5_EMBEDDINGS_DIR / filename
        
        try:
            with open(filepath, 'r') as f:
                data = json.load(f)
            
            loaded_t5_embedding = np.array(data['embedding'])
            loaded_t5_prompt = data.get('prompt', 'Unknown')
            
            print(f"✓ Loaded T5 embedding!")
            print(f"  File: {filename}")
            print(f"  Prompt: '{loaded_t5_prompt}'")
            print(f"  Shape: {loaded_t5_embedding.shape}")
            
        except Exception as e:
            print(f"❌ Error loading T5 embedding: {e}")

load_t5_button.on_click(load_t5_embedding_file)

# ==================== CLIP EMBEDDING WIDGETS ====================

# CLIP embedding selection
clip_file_dropdown = widgets.Dropdown(
    options=[],
    description='CLIP Embedding:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

load_clip_button = widgets.Button(
    description='Load CLIP',
    button_style='success'
)

clip_load_output = widgets.Output()

# Populate CLIP dropdown options (including subfolders)
if CLIP_EMBEDDINGS_DIR.exists():
    clip_files = sorted([str(f.relative_to(CLIP_EMBEDDINGS_DIR)) for f in CLIP_EMBEDDINGS_DIR.glob('**/*.json')])
    clip_file_dropdown.options = clip_files

def load_clip_embedding_file(b):
    global loaded_clip_embedding, loaded_clip_prompt
    
    with clip_load_output:
        clip_load_output.clear_output()
        
        filename = clip_file_dropdown.value
        if not filename:
            print("❌ No file selected!")
            return
        
        filepath = CLIP_EMBEDDINGS_DIR / filename
        
        try:
            with open(filepath, 'r') as f:
                data = json.load(f)
            
            loaded_clip_embedding = np.array(data['embedding'])
            loaded_clip_prompt = data.get('prompt', 'Unknown')
            
            print(f"✓ Loaded CLIP embedding!")
            print(f"  File: {filename}")
            print(f"  Prompt: '{loaded_clip_prompt}'")
            print(f"  Shape: {loaded_clip_embedding.shape}")
            
        except Exception as e:
            print(f"❌ Error loading CLIP embedding: {e}")

load_clip_button.on_click(load_clip_embedding_file)

# ==================== GENERATION CONTROLS ====================

seed_input = widgets.IntText(
    value=42,
    description='Seed:',
    style={'description_width': 'initial'}
)

steps_input = widgets.IntSlider(
    value=4,
    min=1,
    max=50,
    step=1,
    description='Inference Steps:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

width_input = widgets.IntText(
    value=1024,
    description='Width:',
    style={'description_width': 'initial'}
)

height_input = widgets.IntText(
    value=1024,
    description='Height:',
    style={'description_width': 'initial'}
)

generate_button = widgets.Button(
    description='Generate Image',
    button_style='primary',
    layout=widgets.Layout(width='300px', height='50px')
)

generation_output = widgets.Output()

def generate_from_loaded_embeddings(b):
    """
    Generate image using loaded T5 and CLIP embeddings.
    """
    with generation_output:
        generation_output.clear_output()
        
        if 'flux_pipe' not in globals():
            print("❌ FLUX not loaded!")
            return
        
        if loaded_t5_embedding is None:
            print("❌ No T5 embedding loaded! Load a T5 embedding first.")
            return
        
        print(f"Generating image from loaded embeddings...")
        print(f"  Seed: {seed_input.value}")
        print(f"  Inference Steps: {steps_input.value}")
        print(f"  Dimensions: {width_input.value}x{height_input.value}")
        print(f"  T5 embedding shape: {loaded_t5_embedding.shape}")
        print(f"  T5 prompt: '{loaded_t5_prompt}'")
        
        # Convert T5 numpy to torch tensor with bfloat16
        t5_tensor = torch.from_numpy(loaded_t5_embedding.astype(np.float32)).to(
            device=device,
            dtype=torch.bfloat16
        )
        
        # Add batch dimension: [512, 4096] -> [1, 512, 4096]
        t5_tensor = t5_tensor.unsqueeze(0)
        
        # Process CLIP embeddings
        print("\nProcessing CLIP pooled embeddings...")
        
        if loaded_clip_embedding is not None:
            # Use loaded CLIP embedding
            print(f"  Using loaded CLIP embedding")
            print(f"  CLIP embedding shape: {loaded_clip_embedding.shape}")
            print(f"  CLIP prompt: '{loaded_clip_prompt}'")
            
            # Use last token embedding as pooled embedding (EOS token)
            pooled_embeds = torch.from_numpy(
                loaded_clip_embedding[-1:].astype(np.float32)
            ).to(device=device, dtype=torch.bfloat16)
            
        else:
            # Fall back to generating from T5 prompt using CLIP
            print(f"  No CLIP embedding loaded, generating from T5 prompt using CLIP model...")
            
            if loaded_t5_prompt:
                with torch.no_grad():
                    (
                        _,
                        pooled_embeds,
                        _,
                    ) = flux_pipe.encode_prompt(
                        prompt=loaded_t5_prompt,
                        prompt_2=None,
                        device=device,
                        num_images_per_prompt=1,
                        prompt_embeds=None,
                        pooled_prompt_embeds=None,
                        max_sequence_length=512,
                    )
            else:
                print("❌ No CLIP embedding and no prompt available!")
                return
        
        print(f"  Pooled embeddings shape: {pooled_embeds.shape}")
        
        # Construct filename from embeddings
        t5_tokens, t5_manip = parse_embedding_filename(t5_file_dropdown.value)
        clip_tokens, clip_manip = parse_embedding_filename(clip_file_dropdown.value)
        
        # Build filename: t5tokens_t5manip_cliptokens_clipmanip.png
        filename_parts = [t5_tokens]
        if t5_manip:
            filename_parts.append(t5_manip)
        filename_parts.append(clip_tokens)
        if clip_manip:
            filename_parts.append(clip_manip)
        
        filename = '_'.join(filename_parts) + '.png'
        output_filepath = OUTPUT_IMAGES_DIR / filename
        
        # Ensure output directory exists
        output_filepath.parent.mkdir(parents=True, exist_ok=True)
        
        # Generate image
        try:
            print(f"\n{'='*60}")
            print("Running FLUX diffusion...")
            print(f"{'='*60}\n")
            
            image = flux_pipe(
                prompt_embeds=t5_tensor,
                pooled_prompt_embeds=pooled_embeds,
                num_inference_steps=steps_input.value,
                guidance_scale=0.0,
                height=height_input.value,
                width=width_input.value,
                generator=torch.manual_seed(seed_input.value)
            ).images[0]
            
            image.save(output_filepath)
            print(f"✓ Image generated and saved!")
            print(f"  Path: {output_filepath}")
            print(f"  Filename: {filename}\n")
            
            display(image)
            
        except Exception as e:
            print(f"❌ Error generating image: {e}")
            import traceback
            traceback.print_exc()

generate_button.on_click(generate_from_loaded_embeddings)

# ==================== DISPLAY UNIFIED INTERFACE ====================

display(widgets.VBox([
    widgets.HTML("<h2>Embedding Selection & Image Generation</h2>"),
    
    # T5 Embedding Section
    widgets.HTML("<h3>1. T5 Embedding (Required)</h3>"),
    t5_file_dropdown,
    load_t5_button,
    t5_load_output,
    
    # CLIP Embedding Section
    widgets.HTML("<br><h3>2. CLIP Embedding (Optional - auto-generated if not loaded)</h3>"),
    clip_file_dropdown,
    load_clip_button,
    clip_load_output,
    
    # Generation Controls Section
    widgets.HTML("<br><h3>3. Generation Controls</h3>"),
    seed_input,
    steps_input,
    widgets.HTML("<br><b>Image Dimensions:</b>"),
    widgets.HBox([width_input, height_input]),
    widgets.HTML("<br>"),
    generate_button,
    
    # Output Section
    widgets.HTML("<br><h3>Generated Image</h3>"),
    generation_output
]))

## Batch Generation

Generate multiple images with different embedding combinations and organize them into subfolders based on token length:
- `short/` - Generated from short text prompts (encoded at runtime)
- `512_tokens/` - Generated from pre-saved T5 embeddings (512 token sequences)
- `77_tokens/` - Generated from pre-saved CLIP embeddings (77 token sequences)

In [ ]:
# # ==================== BATCH GENERATION WITH SUBFOLDER ORGANIZATION ====================

# # Create output subdirectories
# OUTPUT_SHORT_DIR = OUTPUT_IMAGES_DIR / "short"
# OUTPUT_512_DIR = OUTPUT_IMAGES_DIR / "512_tokens"
# OUTPUT_77_DIR = OUTPUT_IMAGES_DIR / "77_tokens"

# for dir_path in [OUTPUT_SHORT_DIR, OUTPUT_512_DIR, OUTPUT_77_DIR]:
#     os.makedirs(dir_path, exist_ok=True)

# # ==================== T5 EMBEDDING SOURCE SELECTION ====================

# t5_source_radio = widgets.RadioButtons(
#     options=[
#         ('Use T5 embedding file (512 tokens)', '512_tokens'),
#         ('Generate from short prompt', 'short')
#     ],
#     value='512_tokens',
#     description='T5 Source:',
#     style={'description_width': 'initial'},
#     layout=widgets.Layout(width='400px')
# )

# # Short prompt input (for generating T5 at runtime)
# batch_short_prompt = widgets.Text(
#     value='',
#     placeholder='Enter short prompt (e.g., "a red cat")',
#     description='Short Prompt:',
#     style={'description_width': 'initial'},
#     layout=widgets.Layout(width='600px')
# )

# # T5 file selection (for pre-saved embeddings)
# batch_t5_dropdown = widgets.Dropdown(
#     options=[],
#     description='T5 File:',
#     style={'description_width': 'initial'},
#     layout=widgets.Layout(width='600px')
# )

# # Populate T5 dropdown
# if T5_EMBEDDINGS_DIR.exists():
#     t5_files = sorted([str(f.relative_to(T5_EMBEDDINGS_DIR)) for f in T5_EMBEDDINGS_DIR.glob('**/*.json')])
#     batch_t5_dropdown.options = t5_files

# # ==================== CLIP EMBEDDING SOURCE SELECTION ====================

# clip_source_radio = widgets.RadioButtons(
#     options=[
#         ('Use CLIP embedding file (77 tokens)', '77_tokens'),
#         ('Generate from short prompt', 'short'),
#         ('Auto-generate from T5 prompt', 'auto')
#     ],
#     value='77_tokens',
#     description='CLIP Source:',
#     style={'description_width': 'initial'},
#     layout=widgets.Layout(width='400px')
# )

# # Short prompt input for CLIP
# batch_clip_prompt = widgets.Text(
#     value='',
#     placeholder='Enter short prompt for CLIP (e.g., "a red cat")',
#     description='CLIP Prompt:',
#     style={'description_width': 'initial'},
#     layout=widgets.Layout(width='600px')
# )

# # CLIP file selection
# batch_clip_dropdown = widgets.Dropdown(
#     options=[],
#     description='CLIP File:',
#     style={'description_width': 'initial'},
#     layout=widgets.Layout(width='600px')
# )

# # Populate CLIP dropdown
# if CLIP_EMBEDDINGS_DIR.exists():
#     clip_files = sorted([str(f.relative_to(CLIP_EMBEDDINGS_DIR)) for f in CLIP_EMBEDDINGS_DIR.glob('**/*.json')])
#     batch_clip_dropdown.options = clip_files

# # ==================== GENERATION CONTROLS ====================

# batch_seed = widgets.IntText(value=42, description='Seed:', style={'description_width': 'initial'})
# batch_steps = widgets.IntSlider(value=4, min=1, max=50, description='Steps:', style={'description_width': 'initial'}, layout=widgets.Layout(width='400px'))
# batch_width = widgets.IntText(value=1024, description='Width:', style={'description_width': 'initial'})
# batch_height = widgets.IntText(value=1024, description='Height:', style={'description_width': 'initial'})

# batch_generate_button = widgets.Button(
#     description='Generate Batch Image',
#     button_style='primary',
#     layout=widgets.Layout(width='300px', height='50px')
# )

# batch_output = widgets.Output()

# # ==================== VISIBILITY HANDLERS ====================

# def update_t5_visibility(change):
#     if change['new'] == '512_tokens':
#         batch_t5_dropdown.layout.display = 'flex'
#         batch_short_prompt.layout.display = 'none'
#     else:
#         batch_t5_dropdown.layout.display = 'none'
#         batch_short_prompt.layout.display = 'flex'

# def update_clip_visibility(change):
#     if change['new'] == '77_tokens':
#         batch_clip_dropdown.layout.display = 'flex'
#         batch_clip_prompt.layout.display = 'none'
#     elif change['new'] == 'short':
#         batch_clip_dropdown.layout.display = 'none'
#         batch_clip_prompt.layout.display = 'flex'
#     else:  # auto
#         batch_clip_dropdown.layout.display = 'none'
#         batch_clip_prompt.layout.display = 'none'

# t5_source_radio.observe(update_t5_visibility, names='value')
# clip_source_radio.observe(update_clip_visibility, names='value')

# # Initialize visibility
# batch_short_prompt.layout.display = 'none'
# batch_clip_prompt.layout.display = 'none'

# # ==================== BATCH GENERATION FUNCTION ====================

# def determine_output_subfolder(t5_source, clip_source):
#     """
#     Determine output subfolder based on embedding sources.
#     Priority: short > 512_tokens > 77_tokens
#     """
#     if t5_source == 'short' or clip_source == 'short':
#         return OUTPUT_SHORT_DIR, 'short'
#     elif t5_source == '512_tokens':
#         return OUTPUT_512_DIR, '512_tokens'
#     else:
#         return OUTPUT_77_DIR, '77_tokens'

# def sanitize_prompt_for_filename(prompt):
#     """Convert prompt to filename-safe string."""
#     # Replace spaces with underscores, keep only alphanumeric and underscores
#     safe = ''.join(c if c.isalnum() else '_' for c in prompt.lower())
#     # Collapse multiple underscores
#     while '__' in safe:
#         safe = safe.replace('__', '_')
#     return safe.strip('_')[:50]  # Limit length

# def generate_batch_image(b):
#     """Generate image with subfolder organization based on embedding sources."""
#     with batch_output:
#         batch_output.clear_output()
        
#         if 'flux_pipe' not in globals():
#             print("FLUX not loaded!")
#             return
        
#         t5_source = t5_source_radio.value
#         clip_source = clip_source_radio.value
        
#         print(f"T5 source: {t5_source}")
#         print(f"CLIP source: {clip_source}")
        
#         # ==================== PREPARE T5 EMBEDDINGS ====================
        
#         if t5_source == 'short':
#             # Generate T5 embedding from short prompt
#             prompt = batch_short_prompt.value.strip()
#             if not prompt:
#                 print("Please enter a short prompt for T5!")
#                 return
            
#             print(f"\nGenerating T5 embedding from short prompt: '{prompt}'")
            
#             with torch.no_grad():
#                 (
#                     t5_tensor,
#                     _,
#                     _,
#                 ) = flux_pipe.encode_prompt(
#                     prompt=prompt,
#                     prompt_2=None,
#                     device=device,
#                     num_images_per_prompt=1,
#                     prompt_embeds=None,
#                     pooled_prompt_embeds=None,
#                     max_sequence_length=512,
#                 )
            
#             t5_name = sanitize_prompt_for_filename(prompt)
#             t5_prompt_text = prompt
#             print(f"  T5 tensor shape: {t5_tensor.shape}")
            
#         else:
#             # Load from file
#             filename = batch_t5_dropdown.value
#             if not filename:
#                 print("Please select a T5 embedding file!")
#                 return
            
#             filepath = T5_EMBEDDINGS_DIR / filename
#             print(f"\nLoading T5 embedding from: {filename}")
            
#             with open(filepath, 'r') as f:
#                 data = json.load(f)
            
#             t5_embedding = np.array(data['embedding'])
#             t5_prompt_text = data.get('prompt', 'Unknown')
            
#             t5_tensor = torch.from_numpy(t5_embedding.astype(np.float32)).to(
#                 device=device,
#                 dtype=torch.bfloat16
#             ).unsqueeze(0)
            
#             t5_tokens, t5_manip = parse_embedding_filename(filename)
#             t5_name = t5_tokens + ('_' + t5_manip if t5_manip else '')
#             print(f"  T5 tensor shape: {t5_tensor.shape}")
#             print(f"  T5 prompt: '{t5_prompt_text}'")
        
#         # ==================== PREPARE CLIP EMBEDDINGS ====================
        
#         if clip_source == '77_tokens':
#             # Load from file
#             filename = batch_clip_dropdown.value
#             if not filename:
#                 print("Please select a CLIP embedding file!")
#                 return
            
#             filepath = CLIP_EMBEDDINGS_DIR / filename
#             print(f"\nLoading CLIP embedding from: {filename}")
            
#             with open(filepath, 'r') as f:
#                 data = json.load(f)
            
#             clip_embedding = np.array(data['embedding'])
            
#             # Use last token (EOS) as pooled embedding
#             pooled_embeds = torch.from_numpy(
#                 clip_embedding[-1:].astype(np.float32)
#             ).to(device=device, dtype=torch.bfloat16)
            
#             clip_tokens, clip_manip = parse_embedding_filename(filename)
#             clip_name = clip_tokens + ('_' + clip_manip if clip_manip else '')
#             print(f"  Pooled embeddings shape: {pooled_embeds.shape}")
            
#         elif clip_source == 'short':
#             # Generate from short prompt
#             prompt = batch_clip_prompt.value.strip()
#             if not prompt:
#                 print("Please enter a short prompt for CLIP!")
#                 return
            
#             print(f"\nGenerating CLIP pooled embedding from short prompt: '{prompt}'")
            
#             with torch.no_grad():
#                 (
#                     _,
#                     pooled_embeds,
#                     _,
#                 ) = flux_pipe.encode_prompt(
#                     prompt=prompt,
#                     prompt_2=None,
#                     device=device,
#                     num_images_per_prompt=1,
#                     prompt_embeds=None,
#                     pooled_prompt_embeds=None,
#                     max_sequence_length=512,
#                 )
            
#             clip_name = sanitize_prompt_for_filename(prompt)
#             print(f"  Pooled embeddings shape: {pooled_embeds.shape}")
            
#         else:  # auto - generate from T5 prompt
#             print(f"\nAuto-generating CLIP pooled embedding from T5 prompt: '{t5_prompt_text}'")
            
#             with torch.no_grad():
#                 (
#                     _,
#                     pooled_embeds,
#                     _,
#                 ) = flux_pipe.encode_prompt(
#                     prompt=t5_prompt_text,
#                     prompt_2=None,
#                     device=device,
#                     num_images_per_prompt=1,
#                     prompt_embeds=None,
#                     pooled_prompt_embeds=None,
#                     max_sequence_length=512,
#                 )
            
#             clip_name = 'auto'
#             print(f"  Pooled embeddings shape: {pooled_embeds.shape}")
        
#         # ==================== DETERMINE OUTPUT PATH ====================
        
#         output_dir, subfolder_name = determine_output_subfolder(t5_source, clip_source)
        
#         # Build filename
#         filename = f"{t5_name}_{clip_name}.png"
#         output_path = output_dir / filename
        
#         print(f"\nOutput subfolder: {subfolder_name}/")
#         print(f"Output filename: {filename}")
        
#         # ==================== GENERATE IMAGE ====================
        
#         try:
#             print(f"\n{'='*60}")
#             print("Running FLUX diffusion...")
#             print(f"{'='*60}\n")
            
#             image = flux_pipe(
#                 prompt_embeds=t5_tensor,
#                 pooled_prompt_embeds=pooled_embeds,
#                 num_inference_steps=batch_steps.value,
#                 guidance_scale=0.0,
#                 height=batch_height.value,
#                 width=batch_width.value,
#                 generator=torch.manual_seed(batch_seed.value)
#             ).images[0]
            
#             image.save(output_path)
#             print(f"Image saved to: {output_path}")
            
#             display(image)
            
#         except Exception as e:
#             print(f"Error generating image: {e}")
#             import traceback
#             traceback.print_exc()

# batch_generate_button.on_click(generate_batch_image)

# # ==================== DISPLAY BATCH INTERFACE ====================

# display(widgets.VBox([
#     widgets.HTML("<h2>Batch Generation with Subfolder Organization</h2>"),
#     widgets.HTML("<p>Images are saved to subfolders based on embedding source:</p>"),
#     widgets.HTML("<ul><li><b>short/</b> - From short text prompts</li><li><b>512_tokens/</b> - From T5 embedding files</li><li><b>77_tokens/</b> - From CLIP embedding files</li></ul>"),
    
#     widgets.HTML("<h3>T5 Embedding Source</h3>"),
#     t5_source_radio,
#     batch_t5_dropdown,
#     batch_short_prompt,
    
#     widgets.HTML("<br><h3>CLIP Embedding Source</h3>"),
#     clip_source_radio,
#     batch_clip_dropdown,
#     batch_clip_prompt,
    
#     widgets.HTML("<br><h3>Generation Settings</h3>"),
#     batch_seed,
#     batch_steps,
#     widgets.HBox([batch_width, batch_height]),
#     widgets.HTML("<br>"),
#     batch_generate_button,
    
#     widgets.HTML("<br><h3>Output</h3>"),
#     batch_output
# ]))

In [8]:
# Clean up cache to free home directory space
import shutil
import os
from pathlib import Path

def cleanup_cache():
    """Delete cache directories to free up home quota"""
    cache_dirs = [
        Path.home() / "data" / ".cache",
        Path.home() / ".cache" / "huggingface",
        Path.home() / ".cache" / "torch",
        Path.home() / ".cache" / "uv",
    ]
    
    total_freed = 0
    for cache_dir in cache_dirs:
        if cache_dir.exists():
            try:
                # Get size before deletion (approximate)
                size = sum(f.stat().st_size for f in cache_dir.rglob('*') if f.is_file())
                shutil.rmtree(cache_dir)
                total_freed += size
                print(f"\u2713 Deleted {cache_dir}: {size / 1024**3:.2f} GB")
            except Exception as e:
                print(f"\u2717 Could not delete {cache_dir}: {e}")
    
    print(f"\nTotal freed: {total_freed / 1024**3:.2f} GB")

# Run cleanup
cleanup_cache()

✗ Could not delete /home/lauwag/data/.cache: Cannot call rmtree on a symbolic link
✓ Deleted /home/lauwag/.cache/huggingface: 51.00 GB
✓ Deleted /home/lauwag/.cache/uv: 0.05 GB

Total freed: 51.05 GB


---
<sub>Latent Vandalism Workshop • Laura Wagner, 2026 • [laurajul.github.io](https://laurajul.github.io/)</sub>